In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=10,10

import random
import copy
import math
import time

import numpy as np
import pandas as pd 
from sklearn import metrics

from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeRegressor,HoeffdingAdaptiveTreeRegressor
from skmultiflow.drift_detection import PageHinkley


# Helper Functions

In [ ]:
def correction(x):
    for i in range(0,len(x)):
        if x[i]>1:
            x[i]=1
        elif x[i]<0:
            x[i]=0.001
    return x

def rmse(pred,true):
    return(np.sqrt(metrics.mean_squared_error(pred,true)))

def preqeval(model, X, y):
    output = []
    for i in range(0,len(X)):
        output.append(model.predict([X[i]])[0])
        model.partial_fit([X[i]], [y[i]])
    return(output)

           
def nelder(model_list, x_val, y_val):

    model_list.sort(key=lambda x: x['rmse'])
    mid = correction(sum([i['param'] for i in model_list[:-1]])/len(model_list[:-1]))
    
    
    # Reflection
    xr = correction(2*mid - model_list[-1]['param'])
    xr_model = HoeffdingAdaptiveTreeRegressor(split_confidence = xr[0],
                                   learning_ratio_perceptron = xr[1],
                                   learning_ratio_decay = xr[2])
    xr_score = rmse(preqeval(xr_model, x_val, y_val), y_val)

    if xr_score < model_list[2]['rmse']:
        if model_list[0]['rmse'] < xr_score:
            print("1")###############
            model_list[-1]['rmse'] = xr_score
            model_list[-1]['model'] = model_list[-1]['model'].set_params(split_confidence=xr[0],learning_ratio_perceptron=xr[1],learning_ratio_decay=xr[2])
            model_list[-1]['param'] = xr
            return(model_list)
        else:
            # Expansion
            xe = correction(2*xr - mid)
            xe_model = HoeffdingAdaptiveTreeRegressor(split_confidence = xe[0],
                               learning_ratio_perceptron = xe[1],
                               learning_ratio_decay = xe[2])
            xe_score = rmse(preqeval(xe_model, x_val, y_val), y_val)

            if xe_score < xr_score:
                print("exp")####################
                model_list[-1]['rmse'] = xe_score
                model_list[-1]['model'] = model_list[-1]['model'].set_params(split_confidence=xe[0],learning_ratio_perceptron=xe[1],learning_ratio_decay=xe[2])
                model_list[-1]['param'] = xe
                return(model_list)
            else:
                print("ref")#####################
                model_list[-1]['rmse'] = xr_score
                model_list[-1]['model'] = model_list[-1]['model'].set_params(split_confidence=xr[0],learning_ratio_perceptron=xr[1],learning_ratio_decay=xr[2])
                model_list[-1]['param'] = xr
                return(model_list)
    else:
        # Contraction
        if xr_score < model_list[-1]['rmse']:
            print("ref2")####################
            model_list[-1]['rmse'] = xr_score
            model_list[-1]['model'] = model_list[-1]['model'].set_params(split_confidence=xr[0],learning_ratio_perceptron=xr[1],learning_ratio_decay=xr[2])
            model_list[-1]['param'] = xr
            return(model_list)
        else:
            xc = correction((model_list[-1]['param'] + mid)/2)
            xc_model = HoeffdingAdaptiveTreeRegressor(split_confidence = xc[0],
                               learning_ratio_perceptron = xc[1],
                               learning_ratio_decay = xc[2])

            xc_score = rmse(preqeval(xc_model, x_val, y_val), y_val)
            
            if xc_score < model_list[-1]['rmse']:
                print("con")####################
                model_list[-1]['rmse'] = xc_score
                model_list[-1]['model'] = model_list[-1]['model'].set_params(split_confidence=xc[0],learning_ratio_perceptron=xc[1],learning_ratio_decay=xc[2])
                model_list[-1]['param'] = xc
                return(model_list)
            else:
                # Sink
                print("sin")####################
                xs1 = correction((model_list[0]['param'] + model_list[-1]['param'])/2)
                xs_model1 = HoeffdingAdaptiveTreeRegressor(split_confidence = xs1[0],
                               learning_ratio_perceptron = xs1[1],
                               learning_ratio_decay = xs1[2])
                xs_score1 = rmse(preqeval(xs_model1, x_val, y_val), y_val)
                if xs_score1 < model_list[-1]['rmse']:
                    model_list[-1]['rmse'] = xs_score1
                    model_list[-1]['model'] =model_list[-1]['model'].set_params(split_confidence=xs1[0],learning_ratio_perceptron=xs1[1],learning_ratio_decay=xs1[2])
                    model_list[-1]['param'] = xs1

                xs2 = correction((model_list[0]['param'] + model_list[1]['param'])/2)
                xs_model2 = HoeffdingAdaptiveTreeRegressor(split_confidence = xs2[0],
                               learning_ratio_perceptron = xs2[1],
                               learning_ratio_decay = xs2[2])
                xs_score2 = rmse(preqeval(xs_model2, x_val, y_val), y_val)
                if xs_score2 < model_list[-1]['rmse']:
                    model_list[1]['rmse'] = xs_score2
                    model_list[1]['model'] = model_list[1]['model'].set_params(split_confidence=xs2[0],learning_ratio_perceptron=xs2[1],learning_ratio_decay=xs2[2])
                    model_list[1]['param'] = xs2

                xs3 = correction((model_list[0]['param'] + model_list[2]['param'])/2)
                xs_model3 = HoeffdingAdaptiveTreeRegressor(split_confidence = xs3[0],
                               learning_ratio_perceptron = xs3[1],
                               learning_ratio_decay = xs3[2])
                xs_score3 = rmse(preqeval(xs_model3, x_val, y_val), y_val)
                if xs_score3 < model_list[-1]['rmse']:
                    model_list[2]['rmse'] = xs_score3
                    model_list[2]['model'] = model_list[2]['model'].set_params(split_confidence=xs3[0],learning_ratio_perceptron=xs3[1],learning_ratio_decay=xs3[2])
                    model_list[2]['param'] = xs3
                    
                return(model_list)


def draw_graph(values,range_):
    if range_=="no":
        instances = list(range(0,len(values)))
        plt.plot(instances, values)
    else:
        instances = list(range(0,range_[1]-range_[0]))
        plt.plot(instances, values[range_[0]:range_[1]])
        
    plt.title('First run of NM holdout rmse')
    plt.xlabel('instances')
    plt.ylabel('response value')
    plt.show()
    print(np.mean(values))

# DATA FRAME

In [ ]:
gpu = pd.read_csv("sgemm_product.csv")
col = gpu.loc[: , "Run1 (ms)":"Run4 (ms)"]
gpu['run_avg']=col.mean(axis=1)
gpu.drop(col.columns.values.tolist(), axis = 1, inplace = True)

# Start of Algorithm

In [ ]:
stream = DataStream(gpu)
n_samples = 0
max_samples = stream.n_samples


y_pred = []
y_true = []
dynamic_size = 30 
dynamic_counter = -1
X = "a"
ph = PageHinkley()#delta=0.005,threshold=500,,alpha=0.0005
rmse_1000 = []
start = 0
end = 1000
ph_num=0
dynamic_num=0
param = [np.array([random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]),
         np.array([random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]),
         np.array([random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]),
         np.array([random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)])]
models = [{"model":HoeffdingAdaptiveTreeRegressor(split_confidence=param[0][0],learning_ratio_perceptron=param[0][1],learning_ratio_decay=param[0][2]),
             "param":param[0],
             "rmse":1000000,
             "tag":0},
          
          {"model":HoeffdingAdaptiveTreeRegressor(split_confidence=param[1][0],learning_ratio_perceptron=param[1][1],learning_ratio_decay=param[1][2]),
             "param":param[1],
             "rmse":1000000,
             "tag":1},
          
          {"model":HoeffdingAdaptiveTreeRegressor(split_confidence=param[2][0],learning_ratio_perceptron=param[2][1],learning_ratio_decay=param[2][2]),
             "param":param[2],
             "rmse":1000000,
             "tag":2},
          
          {"model":HoeffdingAdaptiveTreeRegressor(split_confidence=param[3][0],learning_ratio_perceptron=param[3][1],learning_ratio_decay=param[3][2]),
             "param":param[3],
             "rmse":1000000,
             "tag":3}]

best = models[0]['model']

In [ ]:
t0 = time.time()
while n_samples < max_samples and stream.has_more_samples():
    t1 = time.time()
    
    X1, y1 = stream.next_sample()
    
    y_true.append(y1[0])
    
    if type(X) == str:
        X = X1
        y = y1
    else:
        X = np.concatenate((X,X1))
        y = np.concatenate((y,y1))
    
    test = best.predict(X1)[0]
    y_pred.append(test)
    best.partial_fit(X1, y1)
    dynamic_counter+=1
    n_samples+=1
    if n_samples>1000:
        rmse_1000.append(rmse(y_true[start:end],y_pred[start:end]))
        start+=1
        end+=1
        ph.add_element(rmse_1000[-1])
        
    if dynamic_size == dynamic_counter:
        dynamic_num+=1
        for i in range(0,4):
            models[i]['rmse'] = rmse(preqeval(models[i]['model'],X,y), y)
        models = nelder(models,X,y)
        sd = np.std(preqeval(models[0]['model'],X,y))
        dynamic_size = round((4*sd**2)/0.95**2)
        if dynamic_size<30:
            dynamic_size=30
            
        dynamic_counter = 0
        
        X = "none"
        y = "none"
        best = models[0]['model']
        t2 = time.time()
        if n_samples>1000:
            print("dynamic:",dynamic_size,"||sample:", n_samples, "||RMSE:",rmse_1000[-1])
        else:
            print("dynamic:",dynamic_size,"||sample:", n_samples)
                  
    # 
    if ph.detected_change():
        ph_num+=1
        for i in range(0,4):
            models[i]['rmse'] = rmse(preqeval(models[i]['model'],X,y), y)
        models = nelder(models,X,y)
        sd = np.std(preqeval(models[0]['model'],X,y))
        dynamic_size = round((4*sd**2)/0.95**2)
        if dynamic_size<30:
            dynamic_size=30

        dynamic_counter = 0

        X = "none"
        y = "none"
        best = models[0]['model']
        t2 = time.time()
        print("PHT||||dynamic:",dynamic_size,"||sample:", n_samples,"||time:", round(t2-t1,2), "||RMSE:",rmse_1000[-1])



                

    